In [ ]:
from os import path
import cognee
import dspy
from dspy.datasets import HotPotQA

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.prune.prune_system()

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

dataset = HotPotQA(
    train_seed = 1,
    train_size = 10,
    eval_seed = 2023,
    dev_size = 0,
    test_size = 0,
    keep_details = True,
)

texts_to_add = []

for train_case in dataset.train:
    train_case_text = "\r\n".join(" ".join(context_sentences) for context_sentences in train_case.get("context")["sentences"])

    texts_to_add.append(train_case_text)

dataset_name = "train_dataset"
await cognee.add(texts_to_add, dataset_name)


In [2]:
from os import path
import cognee

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

print(cognee.datasets.list_datasets())

train_dataset = cognee.datasets.query_data('train_dataset')
print(len(train_dataset))

['train_dataset']
10


In [3]:
from os import path
import cognee

await cognee.prune.prune_system()

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.cognify('train_dataset')

/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=76 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 82472.27it/s]

Processing document (faf8441f-25bf-58d5-a55a-2821e5e40a9e).
Processing document (0a4c678b-77a7-5927-992f-72b81fda2398).
Processing document (ae7c88b9-432a-5f51-a512-885da6f2d1c6).


/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/networkx/classes/reportviews.py:1365: ResourceWarning: unclosed <ssl.SSLSocket fd=79, family=2, type=1, proto=0, laddr=('192.168.9.243', 62357), raddr=('35.245.15.233', 6333)>
  return self.dataview(self, nbunch, data, default=default, keys=keys)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/threading.py:961: ResourceWarning: unclosed <socket.socket fd=83, family=2, type=1, proto=6, laddr=('192.168.9.243', 62358), raddr=('35.245.15.233', 6333)>
  with _active_limbo_lock:
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=83 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 70747.30it/s]
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/pydantic/_in

Document (DOCUMENT__faf8441f-25bf-58d5-a55a-2821e5e40a9e) classified.
Document (DOCUMENT__ae7c88b9-432a-5f51-a512-885da6f2d1c6) classified.
Document (DOCUMENT__0a4c678b-77a7-5927-992f-72b81fda2398) classified.
Document (DOCUMENT__ae7c88b9-432a-5f51-a512-885da6f2d1c6) summarized.
Document (DOCUMENT__faf8441f-25bf-58d5-a55a-2821e5e40a9e) summarized.


/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=85 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=87 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=88 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/pydantic/main.py:1096: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprec

...

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 58254.22it/s]
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:145: ResourceWarning: unclosed <socket.socket fd=93, family=2, type=1, proto=6, laddr=('192.168.9.243', 62398), raddr=('35.245.15.233', 6333)>
  return self.config_dict[name]
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:145: ResourceWarning: unclosed <ssl.SSLSocket fd=96, family=2, type=1, proto=0, laddr=('192.168.9.243', 62399), raddr=('35.245.15.233', 6333)>
  return self.config_dict[name]
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_l

DATA_POINTS
[DataPoint(id='aecc5d09-a5a1-4656-978a-34c0c3fffb03', payload={'value': 'Charlie Babcock', 'references': {'node_id': 'PROPOSITION_NODE__CHARLIE_BABCOCK', 'cognitive_layer': 'COGNITIVE_LAYER__BIOGRAPHICAL_INFORMATION'}}, embed_field='value'), DataPoint(id='d0efe2b8-cc60-41d8-a609-597bc17b45b8', payload={'value': 'Thom Bierdz', 'references': {'node_id': 'PROPOSITION_NODE__THOM_BIERDZ', 'cognitive_layer': 'COGNITIVE_LAYER__BIOGRAPHICAL_INFORMATION'}}, embed_field='value'), DataPoint(id='8dbbe510-da48-43d2-96c0-0f51375d9951', payload={'value': 'Scrubs Season 4', 'references': {'node_id': 'PROPOSITION_NODE__SCRUBS_SEASON_4', 'cognitive_layer': 'COGNITIVE_LAYER__BIOGRAPHICAL_INFORMATION'}}, embed_field='value'), DataPoint(id='6ed1f1b9-60d5-4f2d-9bf9-ca9b1428e170', payload={'value': 'Ernie Grunwald', 'references': {'node_id': 'PROPOSITION_NODE__ERNIE_GRUNWALD', 'cognitive_layer': 'COGNITIVE_LAYER__BIOGRAPHICAL_INFORMATION'}}, embed_field='value'), DataPoint(id='4d2c6b2e-3f3b-468c-

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 54370.61it/s]
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/borisarzentar/nltk_data...
[nltk_data]   Package vader_lexicon is alrea

DATA_POINTS
[DataPoint(id='1a8e288e-cf9a-4d22-a203-03e740097df9', payload={'value': 'Charlie Babcock', 'references': {'node_id': 'PROPOSITION_NODE__CHARLIE_BABCOCK', 'cognitive_layer': 'COGNITIVE_LAYER__CAREER_MILESTONES'}}, embed_field='value'), DataPoint(id='8b938ed0-41eb-4540-9037-85c53b7f3920', payload={'value': 'Desperate Housewives', 'references': {'node_id': 'PROPOSITION_NODE__DESPERATE_HOUSEWIVES', 'cognitive_layer': 'COGNITIVE_LAYER__CAREER_MILESTONES'}}, embed_field='value'), DataPoint(id='e4346ef7-8b3b-4a76-9f33-c6351519d40f', payload={'value': 'Stu', 'references': {'node_id': 'PROPOSITION_NODE__STU', 'cognitive_layer': 'COGNITIVE_LAYER__CAREER_MILESTONES'}}, embed_field='value'), DataPoint(id='83b7fbe2-9ec3-4f51-b5f8-983e100bc655', payload={'value': 'Joely Fisher', 'references': {'node_id': 'PROPOSITION_NODE__JOELY_FISHER', 'cognitive_layer': 'COGNITIVE_LAYER__CAREER_MILESTONES'}}, embed_field='value'), DataPoint(id='9e1b12db-98d1-4b49-beae-0fff78c1a209', payload={'value': 

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 73769.17it/s]


In [ ]:
import networkx as nx
import pandas as pd
import graphistry
from cognee.config import Config
from cognee.utils import render_graph
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

config = Config()
config.load()

graphistry.register(
    api = 3,
    username = config.graphistry_username,
    password = config.graphistry_password
)

graph_client = await get_graph_client(GraphDBType.NETWORKX, "cognee_graph.pkl")
graph = graph_client.graph

edges = nx.to_pandas_edgelist(graph)

nodes_data = [{
    "id": node_id,
    "label": node["name"] if "name" in node else node_id,
} for (node_id, node) in graph.nodes(data = True)]

nodes = pd.DataFrame(nodes_data)

plotter = graphistry.edges(edges, source = "source", destination = "target").nodes(nodes, "id")

plotter.bind(edge_title = "relationship_name", edge_label = "relationship_name", point_title = "name", point_label = "name")
url = plotter.plot(render = False, as_files = True)
print(f"Graph is visualized at: {url}")


In [ ]:
graph = await cognee.cognify(dataset_name)

from cognee.utils import render_graph

await render_graph(graph)

In [ ]:
import cognee
from os import path
from networkx.algorithms import dfs_successors
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

graph_client = await get_graph_client(GraphDBType.NETWORKX)
graph = graph_client.graph

successors_map = dfs_successors(graph, source = "DOCUMENT__0a4c678b-77a7-5927-992f-72b81fda2398")

for parent_id, child_ids in successors_map.items():
    print(graph.nodes[parent_id])
    parent_label = graph.nodes[parent_id]["name"]

    for child_id in child_ids:
        relationship_label = graph[parent_id][child_id].get(0)["relationship_name"]
        child_label = graph.nodes[child_id]["name"]

        print(parent_label + " " + relationship_label + " " + child_label)

In [4]:
from os import path
import cognee
from cognee.modules.search.vector.search_similarity import search_similarity

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

results = await search_similarity("At My Window was released by which American singer-songwriter?")
# results = await search_similarity("At My Window")

print(results)

/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/json/decoder.py:353: ResourceWarning: unclosed <socket.socket fd=97, family=2, type=1, proto=6, laddr=('192.168.9.243', 62412), raddr=('35.245.15.233', 6333)>
  obj, end = self.scan_once(s, idx)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/json/decoder.py:353: ResourceWarning: unclosed <ssl.SSLSocket fd=98, family=2, type=1, proto=0, laddr=('192.168.9.243', 62413), raddr=('35.245.15.233', 6333)>
  obj, end = self.scan_once(s, idx)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/selector_events.py:868: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=96 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 93206.76it/s]
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/pydantic/_internal/_core_utils.py:197: ResourceWarning: unclosed <socket.sock

['Through the Window is an album by the American music project Prurient, the performing name of the artist Dominick Fernow.  The three-song album was released on March 19, 2013 through the English label Blackest Ever Black.  Though released in 2013, the tracks for "Through the Window" were recorded in October 2011 at the same time as Prurient\'s two Hydra Head Records releases — the studio album "Bermuda Drain" (2011) and the EP "Time\'s Arrow" (2011) — and were noted for musically showing more techno influences, akin to one of Fernow\'s other projects, Vatican Shadow.\r\nYes I Am is the fourth studio album by American singer-songwriter Melissa Etheridge, released by Island Records on September 21, 1993 (see 1993 in music).  The title is generally thought to refer to Etheridge\'s recent coming out as a lesbian, confirming long-standing rumors about her personal life.  This is the album that gave Etheridge national recognition.  The rock ballad "Come to My Window" was the first single r

In [5]:
import dsp
import dspy
from dspy.datasets import HotPotQA
from dspy.evaluate.evaluate import Evaluate
from cognee.modules.search.CogneeSearch import CogneeSearch

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

dataset = HotPotQA(
    train_seed = 1,
    train_size = 10,
    eval_seed = 2023,
    dev_size = 0,
    test_size = 0,
    keep_details = True,
)

eval_set = [x.with_inputs('question') for x in dataset.train[:3]]

def evaluate_answer(example, prediction):
    return dsp.answer_match(example.answer, [prediction.answer], frac = 0.8) or \
        dsp.passage_match([example.answer], [prediction.answer])

evaluate = Evaluate(devset = eval_set, metric = evaluate_answer, num_threads = 1, display_progress = True)

program = CogneeSearch()

baseline_eval_score = evaluate(program, devset = eval_set)
print("Baseline eval score: %s", baseline_eval_score)


<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=93, family=2, type=1, proto=6, laddr=('192.168.9.243', 62429), raddr=('35.245.15.233', 6333)>
<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=96, family=2, type=1, proto=6, laddr=('192.168.9.243', 62430), raddr=('35.245.15.233', 6333)>
<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=97, family=2, type=1, proto=6, laddr=('192.168.9.243', 62432), raddr=('35.245.15.233', 6333)>
<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=98, family=2, type=1, proto=6, laddr=('192.168.9.243', 62433), raddr=('35.245.15.233', 6333)>
<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=99, family=2, type=1, proto=6, laddr=('192.168.9.243', 62434), raddr=('35.245.15.233', 6333)>
<frozen importlib._bootstrap_external>:729: ResourceWarning: unclosed <socket.socket fd=10

Context: Through the Window is an album by the American music project Prurient, the performing name of the artist Dominick Fernow.  The three-song album was released on March 19, 2013 through the English label Blackest Ever Black.  Though released in 2013, the tracks for "Through the Window" were recorded in October 2011 at the same time as Prurient's two Hydra Head Records releases — the studio album "Bermuda Drain" (2011) and the EP "Time's Arrow" (2011) — and were noted for musically showing more techno influences, akin to one of Fernow's other projects, Vatican Shadow.
Yes I Am is the fourth studio album by American singer-songwriter Melissa Etheridge, released by Island Records on September 21, 1993 (see 1993 in music).  The title is generally thought to refer to Etheridge's recent coming out as a lesbian, confirming long-standing rumors about her personal life.  This is the album that gave Etheridge national recognition.  The rock ballad "Come to My Window" was the first single r

/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/openai/_client.py:132: ResourceWarning: unclosed <socket.socket fd=82, family=2, type=1, proto=6, laddr=('192.168.9.243', 62627), raddr=('35.245.15.233', 6333)>
  self.models = resources.Models(self)
Average Metric: 1 / 1  (100.0):  33%|███▎      | 1/3 [00:16<00:32, 16.42s/it]

Question: At My Window was released by which American singer-songwriter?
Answer: Townes Van Zandt


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 73035.14it/s]
<frozen genericpath>:19: ResourceWarning: unclosed <socket.socket fd=75, family=2, type=1, proto=6, laddr=('192.168.9.243', 62629), raddr=('35.245.15.233', 6333)>
<frozen genericpath>:19: ResourceWarning: unclosed <socket.socket fd=79, family=2, type=1, proto=6, laddr=('192.168.9.243', 62630), raddr=('35.245.15.233', 6333)>
<frozen genericpath>:19: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('192.168.9.243', 62631), raddr=('35.245.15.233', 6333)>
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 72137.91it/s]
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/base_events.py:782: ResourceWarning: unclosed <socket.socket fd=79, family=2, type=1, proto=6, laddr=('192.168.9.243', 62633), raddr=('35.245.15.233', 6333)>
  handle = events.Handle(callback, args, self, context)
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/asyncio/base_events.py:782: ResourceW

...

Question: which  American actor was Candace Kita  guest starred with 
Answer: Candace Kita guest starred with William James Murray in the FX spoof "Son of the Beach" from 2001 to 2002.


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 44893.16it/s]
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/threading.py:468: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('192.168.9.243', 62643), raddr=('35.245.15.233', 6333)>
  with self._cond:
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/threading.py:468: ResourceWarning: unclosed <socket.socket fd=82, family=2, type=1, proto=6, laddr=('192.168.9.243', 62644), raddr=('35.245.15.233', 6333)>
  with self._cond:
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/threading.py:468: ResourceWarning: unclosed <socket.socket fd=83, family=2, type=1, proto=6, laddr=('192.168.9.243', 62651), raddr=('35.245.15.233', 6333)>
  with self._cond:
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 61294.63it/s]
/Users/borisarzentar/.pyenv/versions/3.11.8/lib/python3.11/selectors.py:74: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('192.1

Context: He Put the Bomp!  In the Bomp is a Greg Shaw tribute album released in November 2007 by Bomp Records in the USA and Vivid Sound Corporation in Japan.  The album features 23 classic songs covered by different bands having in common the raw approach to rock and roll music Shaw most liked.
"Who Put the Bomp (in the Bomp, Bomp, Bomp)" is a doo-wop style hit song from 1961 co-written (with Gerry Goffin) and recorded by Barry Mann.  He was backed up by the Halos, who had previously backed Curtis Lee on "Pretty Little Angel Eyes".  The song was originally released as a single on the ABC-Paramount label (10237).
AMA Manual of Style: A Guide for Authors and Editors is the style guide of the American Medical Association.  It is written by the editors of "JAMA" ("Journal of the American Medical Association") and the "Archives" journals and is most recently published by Oxford University Press.  It specifies the writing and citation styles for use in the journals published by the American

Average Metric: 1 / 3  (33.3): 100%|██████████| 3/3 [00:47<00:00, 15.96s/it]

Question: Which of these publications was most recently published, Who Put the Bomp or Self?
Answer: The most recently published publication is Self.
Average Metric: 1 / 3  (33.3%)
Baseline eval score: %s 33.33
